# Bild-Klassifizierung

Bei der Bild-Klassifizierung identifiziert man das abgebildete Objekt. Objekt-Erkennung kann mithilfe von Bild-Klassifizierung gelöst werden, indem man verschiedene Bildausschnitte klassifizieren lässt. Die mit großer Sicherheit klassifizierten Ausschnitte beinhalten das gesuchte Objekt mit hoher Wahrscheinlichkeit, die anderen können verworfen werden.

In [ ]:
# Imports
import numpy as np
import tensorflow as tf
keras = tf.keras

In [ ]:
# Daten laden
from tensorflow.keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Daten vorverarbeten
def preprocess_image(x):
    # uint8 -> float32
    x = x.astype(np.float32)
    
    # Normalisiere die Bilder (Mean = Null und
    #  Standardabweichung = Eins)
    x = x - x.mean()
    x = x / x.std()
    return x

def preprocess_label(y):
    return y.astype(np.int32)

x_train = preprocess_image(x_train)
y_train = preprocess_label(y_train)

x_test  = preprocess_image(x_test)
y_test  = preprocess_label(y_test)

x_train.mean(), x_train.std(), y_train.shape, y_train.dtype

## Qualitätsmaß

Um zu bewerten, wie gut ein Bildklassifizierungs-Algorithmus funktioniert benötigen wir ein Qualitätsmaß. Dazu wird die Genauigkeit (Accuracy) verwendet. Sie ist definiert als $$Genauigkeit = \frac{Anzahl Richtiger Vorhersagen}{Anzahl Alle Vorhersagen}$$

In [ ]:
def accuracy(labels, predictions):
    # 'labels' beinhaltet die tatsächliche Klasse jedes Bildes
    # 'predictions' beinhaltet unsere vorhergesagte Klasse
    
    # Wir vergleichen die Vorhersage jedes Bildes mit der 
    #   tasächlichen Klasse.
    # Eg. predictions = [1, 2, 3] und labels = [2, 2, 3]
    #   dann ist correct = [False, True, True]
    correct = predictions == labels
    
    # Um damit weiter zu arbeiten müssen wir die Booleans
    #  in Zahlen konvertieren [False, True, True] wird
    #  dadurch [0, 1, 1]
    correct = correct.astype(np.int32)
    
    # Die Anzahl der korrekten Klassifizierungen ist dann
    #   die Summe von 'correct'
    number_correct = np.sum(correct)
    
    # Die Anzahl aller Elemente ist die Länge des Vektors
    total = len(correct)
    
    # Nun können wir die Genauigkeit berechnen:
    return number_correct / total

## Baseline

CIFAR-10 haben wir letztes Tutorial schon kennen gelernt. Die Bilder müssen einer von 10 Klassen zugeordnet werden. Der einfachste Algorithmus ordnet jedem Bild die gleiche Klassen zu.

In [ ]:
# Baseline Klassifikation

# Um für jedes Bild eine Klasse vorherzusagen, müssen wir wissen, wie viele Bilder es gibt:
number_of_testimages = len(y_test)
print("Anzahl der Testbilder: ", number_of_testimages)

predicted_class = 0 # Kann jede Zahl innerhalb von [0, 9] sein
# Unsere Vorhersage is 'predicted class' für jedes der Bilder
y_pred = np.full([number_of_testimages, 1], predicted_class)
print("Baseline Vorhersage: ", y_pred)

In [ ]:
# Die Genauigkeit unsere Baseline Vorhersage
print("Baseline Genauigkeit: ", accuracy(labels=y_test, predictions=y_pred))

## Convolutional Neural Network (CNN)

Convolutional Neural Networks sind die Grundlage vieler aktueller Bildverarbietungsalgorithmen. Keras macht es sehr einfach CNNs zu erstellen: https://keras.io/models/sequential/

In [ ]:
cnn = keras.Sequential(
    [
        # https://keras.io/layers/convolutional/#conv2d
        keras.layers.Conv2D(filters=16, kernel_size=(3,3),
                            strides=(2,2),
                            input_shape=(32, 32, 3)),
        # https://keras.io/layers/normalization/#batchnormalization
        keras.layers.BatchNormalization(),
        # https://keras.io/layers/advanced-activations/#relu
        keras.layers.ReLU(),
        
        # Conv2d -> ReLU -> MaxPooling2D ist ein übliche
        # Art und Weise CNNs zu konstruieren
        # Diese drei werden (nach belieben) wiederholt
        
        keras.layers.Conv2D(filters=32, kernel_size=(3,3),
                            strides=(2,2)),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        keras.layers.MaxPooling2D(),
        
        # Jede Convolution mit stride=(2,2) halbiert die 
        # Bildgröße (von 32x32 auf 16x16 auf 8x8 ... )
        
        keras.layers.Conv2D(filters=64, kernel_size=(3,3), 
                            strides=(2,2)),
        keras.layers.BatchNormalization(),
        keras.layers.ReLU(),
        
        # Wenn die Bildgröße sehr klein ist,
        #   können wir 'flatten' nutzen, um einen
        #   alle übrigen Pixel in einen Vektor zu
        #   reihen. Danach produziert eine Dense
        #   Layer die letztendliche Klassifikation
        
        
        # https://keras.io/layers/core/#flatten
        keras.layers.Flatten(),
        # https://keras.io/layers/core/#dense
        keras.layers.Dense(10),
    ]
)
cnn.summary()

In [ ]:
# Das Cross-Entropy-Loss ist ein weiteres Qualitätsmaß für
#   Neuronal Netze. Je kleiner es ist, desto besser 
#   funktioniert das CNN. Es jedoch keine für menschen 
#   intuitive Einheit, weshalb wir stattdessen die 
#   Genauigkeit verwenden.
# Anders als für die Genauigkeit kann man für die Cross-Entropy
#  jedoch die Ableitung berechnen, was es dem CNN erlaub zu
#  lernen
def loss(y_true, y_pred):
    return tf.losses.sparse_softmax_cross_entropy(tf.cast(y_true, tf.int32), 
                                                  y_pred)

# Der Optmizier berechnet neue Gewichte für das CNN, um das
#   'loss' zu minimieren.
optimizer = tf.train.MomentumOptimizer(learning_rate=0.001, 
                                       momentum=0.9)

# '.compile' gibt dem CNN alles an die Hand, um es zu trainieren
cnn.compile(optimizer=optimizer, 
            loss=loss,
            metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [ ]:
# Das training passiert über die '.fit' Methode, doch zuerst:

# wir können nicht mit allen 50'000 Bildern auf einmal arbeiten,
#  daher teilen wir alle Daten in 'batches' à 32 Bilder und 
#  verarbeiten die Daten batch für batch
batch_size = 32

# Ein Epoch heißt einmal über alle Daten für die Optimierung
#  zu benutzen. Je mehr, desto länger dauert das Training, 
#  aber desto besser kann das CNN werden
epochs = 2

cnn.fit(x=x_train, y=y_train, 
        batch_size=batch_size, 
        epochs=epochs,
        validation_data = (x_test, y_test))

In [ ]:
cnn.evaluate(x_test, y_test)

Dieses Neuronale Netz erreicht 51.62 % Genauigkeit, wir bekommen also jedes zweite Bild korrekt klassifiziert. Es gibt viele Möglichkeiten dies noch weiter zu verbessern.